In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [4]:
df = pd.read_csv('kc_final.csv')
df.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [6]:
df = df[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront']]
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0,221900.0,3,1.00,1180,5650,1.0,0
1,538000.0,3,2.25,2570,7242,2.0,0
2,180000.0,2,1.00,770,10000,1.0,0
3,604000.0,4,3.00,1960,5000,1.0,0
4,510000.0,3,2.00,1680,8080,1.0,0


In [7]:
x = df.drop('price', axis=1)
y = df['price']

In [8]:
# normalização dos dados
min_max_scaler = StandardScaler()
x = min_max_scaler.fit_transform(x)

In [9]:
model = LinearRegression()

In [10]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [21]:
# Armazenará o score de cada fold
mape_score = []

for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model.fit(x_train, y_train)

    y_pred = model.predict(x_val)

    mape = mean_absolute_percentage_error(y_val, y_pred)

    mape_score.append(mape)

mape_score

[0.3439581880058967,
 0.3479761037268207,
 0.34657027263314866,
 0.342947432918732,
 0.34968140400493664]

In [13]:
mape_mean = np.mean(mape_score)
print(f'MAPE médio: {mape_mean}')

MAPE médio: 0.3462266802579069


# Separando 10% dos dados para o teste final

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [15]:
model = LinearRegression()

In [16]:
k=5
kf =KFold(n_splits=5, shuffle=True, random_state=42)

In [19]:
# Armazenará o score de cada fold
mape_score = []

for train_index, val_index in kf.split(x_train, y_train):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model.fit(x_train, y_train)

    y_pred = model.predict(x_val)

    mape = mean_absolute_percentage_error(y_val, y_pred)

    mape_score.append(mape)

mape_score

[0.34269047340638625,
 0.3407415686699184,
 0.3567873608871567,
 0.35806037864171547,
 0.3460030163139594]

In [20]:
mape_mean = np.mean(mape_score)
print(f'MAPE médio: {mape_mean}')

MAPE médio: 0.3488565595838272


In [22]:
y_pred_2 = model.predict(x_test)

In [23]:
mape_test = mean_absolute_percentage_error(y_test, y_pred_2)
mape_test

0.3481842290171099

# Ajustre de Hiperparâmetros

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [33]:
parameters = {'kernel': ('linear', 'rbf'), 'C':[1, 10]}

In [34]:
model = SVR()

In [35]:
clf = GridSearchCV(model, parameters)

In [36]:
clf.fit(x_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SVR()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [1, 10], 'kernel': ('linear', ...)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate parameter ind

In [38]:
df_results = pd.DataFrame(clf.cv_results_)
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.498471,0.150616,0.580380,0.012583,1,linear,"{'C': 1, 'kernel': 'linear'}",0.000528,0.003709,-0.007947,0.007538,-0.000647,0.000636,0.005146,2
1,7.308227,0.131996,3.480627,0.134617,1,rbf,"{'C': 1, 'kernel': 'rbf'}",-0.057075,-0.050609,-0.062816,-0.050625,-0.061084,-0.056442,0.005108,4
2,5.360652,0.095335,0.587728,0.014538,10,linear,"{'C': 10, 'kernel': 'linear'}",0.244411,0.231086,0.228686,0.240853,0.248086,0.238624,0.007531,1
3,7.479312,0.240228,3.422375,0.098341,10,rbf,"{'C': 10, 'kernel': 'rbf'}",-0.033727,-0.027009,-0.038856,-0.024468,-0.031733,-0.031159,0.005063,3


In [39]:
df_results.query("rank_test_score == 1")['params']

2    {'C': 10, 'kernel': 'linear'}
Name: params, dtype: object

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
parameters = {'kernel': ('linear', 'rbf'), 'C':[1, 10]}
model = SVR()
clf = GridSearchCV(model, parameters, cv=10)
clf.fit(x_train, y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SVR()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [1, 10], 'kernel': ('linear', ...)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",10
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate parameter index

In [42]:
df_results2 = pd.DataFrame(clf.cv_results_)
df_results2

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,6.780799,0.256082,0.338499,0.048361,1,linear,"{'C': 1, 'kernel': 'linear'}",0.004770,0.010875,0.004606,0.017027,0.002252,-0.001796,0.013726,0.011313,0.006692,0.007618,0.007708,0.005343,2
1,9.343016,0.318456,1.901789,0.105919,1,rbf,"{'C': 1, 'kernel': 'rbf'}",-0.058905,-0.054767,-0.050602,-0.052024,-0.055995,-0.069447,-0.047026,-0.055026,-0.059579,-0.061658,-0.056503,0.005996,4
2,6.384779,0.256959,0.311340,0.014066,10,linear,"{'C': 10, 'kernel': 'linear'}",0.255954,0.265722,0.226108,0.276440,0.236323,0.258242,0.248218,0.262238,0.260444,0.264244,0.255393,0.014077,1
3,9.352635,0.140623,1.927148,0.163578,10,rbf,"{'C': 10, 'kernel': 'rbf'}",-0.033805,-0.027594,-0.029139,-0.019111,-0.031995,-0.036060,-0.019365,-0.026617,-0.028549,-0.026126,-0.027836,0.005245,3


In [43]:
print(clf.best_params_)

{'C': 10, 'kernel': 'linear'}
